In [2]:
from ultralytics import YOLO

# Load YOLOv8 model
model = YOLO("yolov8s.pt")  # Change to 'yolov8s.pt' for better accuracy

# Train the model
model.train(
    data="C:\\Users\\kayma\\Downloads\\Bar_Path_Tracking.v2i.yolov8\\data.yaml",
    epochs=20, 
    imgsz=416, 
    batch=4, 
    device="cpu"  # Use "cpu" if no GPU
)


Ultralytics 8.3.91  Python-3.9.21 torch-2.6.0+cpu CPU (13th Gen Intel Core(TM) i5-13600KF)
engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=C:\Users\kayma\Downloads\Bar_Path_Tracking.v2i.yolov8\data.yaml, epochs=20, time=None, patience=100, batch=4, imgsz=416, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, 

train: Scanning C:\Users\kayma\Downloads\Bar_Path_Tracking.v2i.yolov8\train\labels... 770 images, 6 backgrounds, 0 corrupt: 100%|██████████| 770/770 [00:00<00:00, 1247.79it/s]

train: New cache created: C:\Users\kayma\Downloads\Bar_Path_Tracking.v2i.yolov8\train\labels.cache



val: Scanning C:\Users\kayma\Downloads\Bar_Path_Tracking.v2i.yolov8\valid\labels... 72 images, 0 backgrounds, 0 corrupt: 100%|██████████| 72/72 [00:00<00:00, 1145.78it/s]

val: New cache created: C:\Users\kayma\Downloads\Bar_Path_Tracking.v2i.yolov8\valid\labels.cache


...

                   all         72         72          1      0.985      0.994       0.73



20 epochs completed in 0.529 hours.
Optimizer stripped from runs\detect\train\weights\last.pt, 22.5MB
Optimizer stripped from runs\detect\train\weights\best.pt, 22.5MB

Validating runs\detect\train\weights\best.pt...
Ultralytics 8.3.91  Python-3.9.21 torch-2.6.0+cpu CPU (13th Gen Intel Core(TM) i5-13600KF)
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:01<00:00,  4.80it/s]


                   all         72         72          1      0.985      0.994      0.731
Speed: 0.1ms preprocess, 23.0ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to runs\detect\train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000019F9F084220>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

In [ ]:
import cv2
import os
import mediapipe as mp
from ultralytics import YOLO

# Set the video path
video_path = "barbell_video.mp4"  # Change this to your actual video filename

# Check if the video file exists before proceeding
if not os.path.exists(video_path):
    print(f"❌ ERROR: Video file '{video_path}' not found! Please upload a valid video.")
    exit()  # Exit the program to prevent creating an empty output video

# Load the trained YOLOv8 model
model = YOLO("runs/detect/train/weights/best.pt")

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Open the video file
cap = cv2.VideoCapture(video_path)

# Get video properties
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define output video settings
output_path = "barpath_output.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

# Only create the output file if the input video opens successfully
if not cap.isOpened():
    print(f"❌ ERROR: Could not open video file '{video_path}'. Exiting...")
    exit()

out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

# List to store barbell center positions
bar_path_points = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to RGB for MediaPipe
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Run YOLO on the frame to detect the barbell
    results = model(frame)

    # Run MediaPipe Pose on the frame to detect the body
    pose_results = pose.process(rgb_frame)

    # Draw YOLO detections (Barbell)
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # Bounding box coordinates
            conf = float(box.conf[0])  # Confidence score
            class_id = int(box.cls[0])

            # Calculate the center of the barbell
            center_x = int((x1 + x2) / 2)
            center_y = int((y1 + y2) / 2)

            # Store the barbell center position
            bar_path_points.append((center_x, center_y))

            # Draw bounding box around detected barbell
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

            # Display class label and confidence
            label = f"Barbell ({conf:.2f})"
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Draw Pose Landmarks if detected
    if pose_results.pose_landmarks:
        mp_drawing.draw_landmarks(frame, pose_results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2, circle_radius=2))

    # Draw the bar path as a straight line
    if len(bar_path_points) > 1:
        cv2.line(frame, bar_path_points[0], bar_path_points[-1], (0, 0, 255), 3)  # Red line

    # Write the frame to the output video
    out.write(frame)

    # Show the frame
    cv2.imshow("Bar Path Tracking with MediaPipe Pose", frame)

    # Exit on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()



0: 256x416 (no detections), 46.1ms
Speed: 1.4ms preprocess, 46.1ms inference, 0.2ms postprocess per image at shape (1, 3, 256, 416)

0: 256x416 (no detections), 41.2ms
Speed: 0.7ms preprocess, 41.2ms inference, 0.2ms postprocess per image at shape (1, 3, 256, 416)

0: 256x416 (no detections), 45.4ms
Speed: 0.7ms preprocess, 45.4ms inference, 0.2ms postprocess per image at shape (1, 3, 256, 416)

0: 256x416 (no detections), 43.8ms
Speed: 0.6ms preprocess, 43.8ms inference, 0.2ms postprocess per image at shape (1, 3, 256, 416)

0: 256x416 (no detections), 43.8ms
Speed: 0.7ms preprocess, 43.8ms inference, 0.2ms postprocess per image at shape (1, 3, 256, 416)

0: 256x416 (no detections), 42.0ms
Speed: 0.6ms preprocess, 42.0ms inference, 0.2ms postprocess per image at shape (1, 3, 256, 416)

0: 256x416 (no detections), 42.6ms
Speed: 0.6ms preprocess, 42.6ms inference, 0.2ms postprocess per image at shape (1, 3, 256, 416)

0: 256x416 (no detections), 41.2ms
Speed: 0.7ms preprocess, 41.2ms i

KeyboardInterrupt: 

: 